# Observed Air Quality (PurpleAir)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and saves the results as a csv file.

Documentation is available here: https://api.purpleair.com.
You can read this article for help getting started: https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180.

From PurpleAir: 

"The data from individual sensors will update no less than every 30 seconds. As a courtesy, we ask that you limit the number of requests to no more than once every 1 to 10 minutes, assuming you are only using the API to obtain data from sensors. If retrieving data from multiple sensors at once, please send a single request rather than individual requests in succession.

The PurpleAir historical API is released as of July 18, 2022. For more information, view this post: https://community.purpleair.com/t/new-version-of-the-purpleair-api-on-july-18th/1251.

Please let us know if you have any questions or concerns, and have a great day!"

A paper on this process: https://doi.org/10.5194/amt-14-4617-2021 (Link for [Download](https://www.researchgate.net/publication/352663348_Development_and_application_of_a_United_States-wide_correction_for_PM25_data_collected_with_the_PurpleAir_sensor) )

Chat on which PM Estimate to use: https://community.purpleair.com/t/pm2-5-algorithms/3972/6

In [45]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects
import time # For sleep in for loop

# Analysis

import numpy as np
import pandas as pd
import geopandas as gpd

# Current working directory

cwd = os.getcwd()

## Definitions

In [46]:
spike_threshold = 28 # Micgrograms per meter cubed

In [47]:
# This is my personal API key... Please use responsibly! 51592903-B445-11ED-B6F4-42010A800007

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


### Load Sensor Information

In [48]:
# Sensor Indices (from City of Minneapolis)

datapath = os.path.join(cwd, '..', '..', '..', 'Data')

sensor_info = gpd.read_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

### Summary Statistics Functions

In [49]:
%run Summary_Functions.py

print('Stat Names:\n\n', summary_stats_names, '\n')
print('Stat Types:\n\n',summary_stats_dtypes, '\n')

print('Function Names:\n\n', summary_stats_functions)

Stat Names:

 ['n_observations', 'humidity_fullDay_mean', 'temperature_fullDay_mean', 'pressure_fullDay_mean', 'pm25_fullDay_mean', 'pm25_fullDay_min', 'pm25_fullDay_minTime', 'pm25_fullDay_max', 'pm25_fullDay_maxTime', 'pm25_fullDay_std', 'pm25_fullDay_minutesAbove12ug', 'pm25_morningRush_mean', 'pm25_morningRush_min', 'pm25_morningRush_minTime', 'pm25_morningRush_max', 'pm25_morningRush_maxTime', 'pm25_morningRush_std', 'pm25_eveningRush_mean', 'pm25_eveningRush_min', 'pm25_eveningRush_minTime', 'pm25_eveningRush_max', 'pm25_eveningRush_maxTime', 'pm25_eveningRush_std', 'pm25_daytimeAmbient_mean', 'pm25_daytimeAmbient_min', 'pm25_daytimeAmbient_minTime', 'pm25_daytimeAmbient_max', 'pm25_daytimeAmbient_maxTime', 'pm25_daytimeAmbient_std', 'pm25_nighttimeAmbient_mean', 'pm25_nighttimeAmbient_min', 'pm25_nighttimeAmbient_minTime', 'pm25_nighttimeAmbient_max', 'pm25_nighttimeAmbient_maxTime', 'pm25_nighttimeAmbient_std'] 

Stat Types:

 [<class 'int'>, <class 'float'>, <class 'float'>, <

### Functions

In [50]:
# QAQC

def qaqc(df):
    '''This function wil perform some basic QAQC
    '''
    
    clean_df = df.copy()
    
    # Convert timestamp to datetime
    
    clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'], unit='s')
    
    # Remove obvious error values
    
    clean_df = clean_df[clean_df.pm25 < 1000] 
    
    # Remove NaNs
    
    clean_df = clean_df.dropna()
    
    return clean_df

# Remove and record Spikes

def get_spikes(df, spike_threshold):
    '''This function removes spikes from a dataframe 
    and returns both the new dataframe
    and a separate spike dataframe
    '''
    
    df_w_spikes = df.copy()
    
    condition = (df.pm25 > spike_threshold)
    
    df_w_spikes['is_spike'] = condition
    
    spikes = df_w_spikes[condition].copy()
    
    return df_w_spikes, spikes

# Get Summary Stats

def get_summary_stats(df):
    ''' This is the main function. It will run all of our functions that get summary stats
    and return as a list.
    '''
    
    stats = []
    
    # Run the functions
    
    for f in summary_stats_functions:
        stats += f(df)
    
    return stats

### Set Up Parameters for Query

In [51]:
### Query Strings

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=10'

# Environmental fields

env_fields = ['humidity', 'temperature', 'pressure', 'pm2.5_atm']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

# My Header

my_headers = {'X-API-Key': api}

## The Loop

In [52]:
## Iterables

# Dates

# first_date = pd.to_datetime(sensors_df.date_created, unit = 's').min()
# ^ This is just untrue...


first_date = dt.datetime(2022, 6, 15) # June 15th, 2022?

datelist = pd.date_range(start = first_date, 
                         end = dt.datetime.today(),
                        normalize = True)

print('Last Run on ', dt.datetime.today())

Last Run on  2023-06-27 19:38:28.893911


In [53]:
## Initialize Storage

# If this script has already been run, please replace the dataframes with their respective .csvs

has_run = True

if has_run:
    
    daily_summary_df = pd.read_csv(os.path.join(datapath, 'daily_summaries.csv'))
    
    daily_summary_no_spikes_df = pd.read_csv(os.path.join(datapath, 'daily_summaries_no_spikes.csv'))
    
    all_spikes_df = pd.read_csv(os.path.join(datapath, 'all_spikes.csv'))
    all_spikes_df['timestamp'] = pd.to_datetime(all_spikes_df.timestamp)
    
    no_data = pd.read_csv(os.path.join(datapath, 'no_data_sensors.csv'))
                                   
    
else: # Initialize the daily summary dataframes
    
    # Daily Summary

    cols = ['sensor_index', 'date'] + summary_stats_names

    datatypes = [int, str] + summary_stats_dtypes

    dtypes = np.dtype(list(zip(cols, datatypes)))

    daily_summary_df = pd.DataFrame(np.empty(0, dtype = dtypes))

    # Daily Summary (No Spikes)

    cols = ['sensor_index', 'date'] + summary_stats_names

    datatypes = [int, str] + summary_stats_dtypes

    dtypes = np.dtype(list(zip(cols, datatypes)))

    daily_summary_no_spikes_df = pd.DataFrame(np.empty(0, dtype = dtypes))

    # Spikes

    all_spikes_df = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                                      ('timestamp', pd._libs.tslibs.timestamps.Timestamp),
                                                      ('pm25', float)]
                                     )
                            )

    # No Data for sensor

    no_data = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                                ('date', str)
                                               ]))

In [54]:
# Iterate through the days

for i in range(len(datelist)-1):
    
    # Set up Timestamp for query    
    
    start_timestamp = int(datelist[i].timestamp())
    end_timestamp = int(datelist[i+1].timestamp())
    
    time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)
    
    # Select Sensors that had been created before this date
    
    select_sensors = sensor_info[sensor_info.date_created.astype(int)//10**9 <= start_timestamp]
    
    sensor_ids = select_sensors.sensor_index
    
    # Iterate through the Sensors
    
    for sensor_id in sensor_ids:
        
        # For skipping to last spot
        
        is_done = (daily_summary_df.sensor_index == int(sensor_id)) & (daily_summary_df.date == str(datelist[i].date()))
        is_no_data = (no_data.sensor_index == int(sensor_id)) & (no_data.date == str(datelist[i].date()))
        
        # If either of these has a true, it has been parsed
        # True is not in both, then we should process
        if (True not in is_done.values) & (True not in is_no_data.values):
            
            ### Actual Loop

            time.sleep(3)

            # Base URL
            base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

            # Put it all together
            query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

            response = requests.get(query_url, headers=my_headers)

            if response.status_code == 200:

                # Read response as CSV data
                csv_data = response.content.decode('utf-8')

                if csv_data.count('\n') == 1: # There is only one line (empty data)
                    # print(f"No data for sensor {sensor_id} on {datelist[i]}")

                    no_data.loc[len(no_data.index)] = [sensor_id, str(datelist[i].date())]

                else:
                    # Parse CSV data into pandas DataFrame
                    df_individual_sensor = pd.read_csv(io.StringIO(csv_data),
                                                       header=0
                                                      )[['time_stamp', 'humidity', 'temperature', 
                                                         'pressure', 'pm2.5_atm']]

                    df_individual_sensor.columns = ['timestamp','humidity', 'temperature', 
                                                    'pressure', 'pm25']

                    # Perform QAQC

                    clean = qaqc(df_individual_sensor)

                    # Remove Spikes & Concatenate to main storage of spikes

                    clean_w_spikes, spikes = get_spikes(clean, spike_threshold)

                    spikes['sensor_index'] = int(sensor_id)

                    all_spikes_df = pd.concat([all_spikes_df, 
                                               spikes[['sensor_index',
                                                        'timestamp',
                                                        'pm25']]
                                              ],
                                               ignore_index=True)

                    # Get Stats (With Spikes)

                    sum_stats = get_summary_stats(clean_w_spikes)

                    # Add to the daily summary dataframe

                    row = [int(sensor_id), str(datelist[i].date())] + sum_stats

                    daily_summary_df.loc[len(daily_summary_df.index)] = row

                    # Get Stats (Without Spikes)

                    no_spikes = clean_w_spikes[clean_w_spikes.is_spike == False]

                    sum_stats = get_summary_stats(no_spikes)

                    # Add to the daily summary dataframe

                    row = [int(sensor_id), str(datelist[i].date())] + sum_stats

                    daily_summary_no_spikes_df.loc[len(daily_summary_no_spikes_df.index)] = row

            else:
                print(f"Error fetching data for sensor {sensor_id}: {response.status_code} on {datelist[i].date()}")
    
    
    # Save it!?! After a day is processed, yes!

    daily_summary_df.to_csv('daily_summaries.csv', index = False)

    daily_summary_no_spikes_df.to_csv('daily_summaries_no_spikes.csv', index = False)
    
    # For testing a week
    
    # if i == 7:
    #     break

In [55]:
daily_summary_df.tail(10)

# Drop Duplicates
# daily_summary_df = daily_summary_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
14340,157787,2023-06-26,31,68.760742,73.081710,978.646484,13.94629,7.429,00:10:00,24.677,...,None,None,None,None,10.830237,7.429,00:10:00,15.745,02:50:00,2.519632
14341,157785,2023-06-26,143,57.904895,75.203497,980.168993,22.466727,6.593,10:50:00,54.94,...,14:50:00,19.631,12:50:00,3.775702,12.441579,9.181,00:10:00,18.591,03:00:00,2.409537
14342,157837,2023-06-26,144,53.550000,77.376389,979.890792,21.997292,5.64,11:00:00,54.066,...,14:50:00,17.242,12:50:00,3.504537,13.487526,9.194,00:40:00,27.218,00:10:00,4.427722
14343,157845,2023-06-26,0,NaN,NaN,NaN,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14344,157861,2023-06-26,144,55.313889,76.434722,979.991069,21.927701,6.652,11:00:00,53.624,...,14:50:00,21.172,12:50:00,3.662033,13.629947,8.084,00:10:00,18.977,03:00:00,2.769023
14345,157871,2023-06-26,0,NaN,NaN,NaN,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14346,157877,2023-06-26,67,50.111940,78.635075,982.389164,28.976485,4.299,00:10:00,52.228,...,14:40:00,24.815,13:30:00,5.000163,8.610875,4.299,00:10:00,13.503,00:00:00,3.864628
14347,157935,2023-06-26,144,100.000000,375.000000,-250.010000,19.591142,4.225,11:00:00,50.304,...,14:50:00,15.874,13:00:00,2.793701,10.335316,7.338,00:40:00,17.254,03:00:00,2.576219
14348,166459,2023-06-26,144,63.716667,76.044444,981.218125,23.804306,6.309,00:10:00,165.526,...,14:50:00,23.819,12:40:00,4.210509,11.760158,6.309,00:10:00,16.385,03:00:00,2.371548
14349,168327,2023-06-26,144,54.531944,79.225000,979.596875,20.013944,6.669,10:40:00,49.303,...,14:50:00,18.257,12:50:00,3.037315,10.754895,7.471,00:00:00,16.299,03:00:00,2.142776


In [56]:
daily_summary_no_spikes_df.tail(10)

# To Remove Duplicates
# daily_summary_no_spikes_df = daily_summary_no_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
14340,157787,2023-06-26,31,68.760742,73.081710,978.646484,13.94629,7.429,00:10:00,24.677,...,None,None,None,None,10.830237,7.429,00:10:00,15.745,02:50:00,2.519632
14341,157785,2023-06-26,105,66.565714,72.040952,979.567410,14.611562,6.593,10:50:00,27.197,...,14:50:00,19.631,12:50:00,3.775702,12.441579,9.181,00:10:00,18.591,03:00:00,2.409537
14342,157837,2023-06-26,106,61.307547,73.871698,979.276698,14.099962,5.64,11:00:00,27.759,...,14:50:00,17.242,12:50:00,3.504537,13.487526,9.194,00:40:00,27.218,00:10:00,4.427722
14343,157845,2023-06-26,0,NaN,NaN,NaN,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14344,157861,2023-06-26,109,63.467890,72.671560,979.449174,14.84411,6.652,11:00:00,27.218,...,14:50:00,21.172,12:50:00,3.662033,13.629947,8.084,00:10:00,18.977,03:00:00,2.769023
14345,157871,2023-06-26,0,NaN,NaN,NaN,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14346,157877,2023-06-26,34,64.038235,73.569118,982.056059,15.677485,4.299,00:10:00,25.875,...,14:40:00,24.815,13:30:00,5.000163,8.610875,4.299,00:10:00,13.503,00:00:00,3.864628
14347,157935,2023-06-26,109,100.000000,375.000000,-250.010000,12.518505,4.225,11:00:00,27.032,...,14:50:00,15.874,13:00:00,2.793701,10.335316,7.338,00:40:00,17.254,03:00:00,2.576219
14348,166459,2023-06-26,109,71.655046,72.321101,980.671138,15.057055,6.309,00:10:00,27.284,...,14:50:00,23.819,12:40:00,4.210509,11.760158,6.309,00:10:00,16.385,03:00:00,2.371548
14349,168327,2023-06-26,109,63.855046,73.515596,979.024459,13.175578,6.669,10:40:00,26.839,...,14:50:00,18.257,12:50:00,3.037315,10.754895,7.471,00:00:00,16.299,03:00:00,2.142776


In [57]:
all_spikes_df

# To Remove Duplicates
# all_spikes_df = all_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,timestamp,pm25
0,143226,2022-06-17 22:10:00,42.698
1,143226,2022-06-17 22:20:00,28.325
2,143226,2022-06-17 22:00:00,213.712
3,142720,2022-06-18 03:20:00,30.292
4,142720,2022-06-18 03:30:00,33.261
...,...,...,...
252149,168327,2023-06-26 18:20:00,30.272
252150,168327,2023-06-26 19:30:00,36.652
252151,168327,2023-06-26 20:30:00,39.044
252152,168327,2023-06-26 20:00:00,33.369


In [58]:
# no_data['date'] = pd.to_datetime(no_data.date).dt.date.astype(str) # If need to correct the dates
no_data
# To Remove Duplicates
# no_data = no_data.drop_duplicates(ignore_index = True).copy()

,sensor_index,date
0,142718,2022-06-15
1,142720,2022-06-15
2,142726,2022-06-15
3,142718,2022-06-15
4,142720,2022-06-15
...,...,...
8616,143634,2023-06-26
8617,143648,2023-06-26
8618,145504,2023-06-26
8619,145610,2023-06-26


In [59]:
daily_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14350 entries, 0 to 14349
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   sensor_index                   14350 non-null  int64  
 1   date                           14350 non-null  object 
 2   n_observations                 14350 non-null  int64  
 3   humidity_fullDay_mean          13553 non-null  float64
 4   temperature_fullDay_mean       13553 non-null  float64
 5   pressure_fullDay_mean          13553 non-null  float64
 6   pm25_fullDay_mean              13553 non-null  object 
 7   pm25_fullDay_min               13553 non-null  object 
 8   pm25_fullDay_minTime           13553 non-null  object 
 9   pm25_fullDay_max               13553 non-null  object 
 10  pm25_fullDay_maxTime           13553 non-null  object 
 11  pm25_fullDay_std               13533 non-null  object 
 12  pm25_fullDay_minutesAbove12ug  14350 non-null 

In [60]:
# # Save a test dataframe

# clean_w_spikes.to_csv('example_df.csv', index = False)

# Save it!?!

savepath = os.path.join('..', '..','..', 'Data')

daily_summary_df.to_csv(os.path.join(savepath, 'daily_summaries.csv'), index = False)

daily_summary_no_spikes_df.to_csv(os.path.join(savepath, 'daily_summaries_no_spikes.csv'), index = False)

all_spikes_df.to_csv(os.path.join(savepath, 'all_spikes.csv'), index = False)

no_data.to_csv(os.path.join(savepath, 'no_data_sensors.csv'), index = False)